In [ ]:
%%time
import dask
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML
from cartopy import crs, feature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import numpy as np
import pymannkendall as mk
import os
# from cdo import *
# cdo = Cdo()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path='/media/kenz/1B8D1A637BBA134B/CHIRTS'

In [ ]:
tmax = 'Tmax/chirts.Tmax.1983.2016.WA.days_p25.nc'
tmin = 'Tmin/chirts.Tmin.1983.2016.WA.days_p25.nc'

tmax90 = 'Tmax/chirts.Tmax.1983.2016.WA.days_p25_ymonpctl.nc'
tmin90 = 'Tmin/chirts.Tmin.1983.2016.WA.days_p25_ymonpctl.nc'

tmean_ehf = 'Tmean/ehf.nc'

tmean = 'Tmean/chirts.Tmean.1983.2016.WA.days_p25.nc'
# tmean95 = 'Tmean/chirts.clim.Tmean95.1983.2016.WA.days_p25.nc'
# Tmean/chirts.clim.Tmean95.1983.2016.WA.days_p25.nc

# tmean_accl = 'Tmean/accl.nc'
# tmean_sig = 'Tmean/sig95.nc'

In [ ]:
def show_axes():
    for i,j in enumerate(ax):
        ax[i].set_extent([-15,10,0,15])
        ax[i].add_feature(feature.COASTLINE)
        ax[i].add_feature(feature.BORDERS)
        ax[i].add_feature(feature.STATES, linewidth = 0.2)
        ax[i].set_xticks([-10,-5,0,5], crs=crs.PlateCarree())
        ax[i].set_yticks([5,10], crs=crs.PlateCarree())
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax[i].xaxis.set_major_formatter(lon_formatter)
        ax[i].yaxis.set_major_formatter(lat_formatter)
        
def set_fig_params(axes):
    for i,j in enumerate(axes):
    # for i in (range(0,len(axes))):
        ax[i].set_extent([-19,15,4,24])
        ax[i].add_feature(feature.COASTLINE)
        ax[i].add_feature(feature.BORDERS)
        ax[i].add_feature(feature.STATES, linewidth = 0.2)
        ax[i].set_xticks([-20,-10,0,10], crs=crs.PlateCarree())
        ax[i].set_yticks([5,10,15,20], crs=crs.PlateCarree())
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax[i].xaxis.set_major_formatter(lon_formatter)
        ax[i].yaxis.set_major_formatter(lat_formatter)
        
def CTpctl(path,temp,pctl,var):
    temp = df_read(path,temp,var)
    mask = temp.groupby('time.month') > pctl
    return temp.where(mask, drop=True)

def df_read(path,file,var):
    ds = xr.open_dataset(f'{path}/{file}')[f'{var}'].sortby('time').drop_duplicates('time').sel(time=slice('1983',None))
    return ds

def sellonlat(data):
    return data.sel(longitude=10,latitude=10,method='nearest')

def selone(data):
    return data.sel(time='2010')

def rising_filter(array, axis):

    # Make sure there are enough points
    assert(array.shape[axis] == 5)
    # Make sure we're working on the last axis
    assert(axis == array.ndim-1 or axis == -1)
    
    left = array[..., 1]
    right = array[..., 2:].sum(axis=axis)

    return np.logical_and(np.isnan(left), np.isfinite(right))

def rising_filter_dask(x, dim):

    return xr.apply_ufunc(rising_filter, x, input_core_dims=[[dim]],
                             kwargs={'axis': -1},
                             dask='parallelized',
                             output_dtypes=[bool])
def sens_slopes(data):
    data = data.sel(longitude=np.arange(-19.875, 21.875,0.25), latitude=np.arange(3.125,26.875,0.25), method = 'nearest')
    output=[]
    for i in np.arange(len(data.latitude.values)):
        for j in np.arange(len(data.longitude.values)):
            try:
                slope_val = mk.sens_slope(data[:,i,j]).slope
            except:
                slope_val = np.nan
            output.append(slope_val)

    output = np.copy(output).reshape(data.latitude.size,data.longitude.size)
    slopes=xr.DataArray(output, dims=('latitude','longitude'), coords={'latitude':data.latitude,'longitude':data.longitude})
    return slopes

def HWD(data):
    windows = data.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
    heatwave_starts = rising_filter_dask(windows, dim='rolling_dim')
    return heatwave_starts

In [ ]:
tx90 = df_read(path,tmax90,'Tmax')
tx90 = tx90.groupby('time.month').mean('time')

tn90 = df_read(path,tmin90,'Tmin')
tn90 = tn90.groupby('time.month').mean('time')

In [ ]:
ctn90 = CTpctl(path,tmin,tn90,'Tmin')

In [ ]:
ctx90 = CTpctl(path,tmax,tx90,'Tmax')

In [ ]:
windows = ctx90.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
hwn_tx = rising_filter_dask(windows, dim='rolling_dim')

windows = ctn90.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
hwn_tn = rising_filter_dask(windows, dim='rolling_dim')

In [ ]:
yearly_av_hw_tx = hwn_tx.sum('time')/33
lenght_av_hw_tx = hwn_tx.groupby('time.year').sum('time').mean('year')
hwf_tx = yearly_av_hw_tx*lenght_av_hw_tx

In [ ]:
yearly_av_hw_tn = hwn_tn.sum('time')/33
lenght_av_hw_tn = hwn_tn.groupby('time.year').sum('time').mean('year')
hwf_tn = yearly_av_hw_tn*lenght_av_hw_tn

In [ ]:
ehf = df_read(path,tmean_ehf,'Tmax')
ds_gtehf = ehf.groupby('time.year').count()

#### Analysis for Maximum Temperature

In [ ]:
######## Plotting Tmax 90 ########
fig, axes = plt.subplots(ncols = 4, nrows=3, figsize = (12,8))
ax=axes.flatten()
months = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG', 'SEP','OCT','NOV','DEC']
for i, month in enumerate(tx90['month']):
    cb = tx90.sel(month=month).plot(ax=ax[i], add_colorbar = False, cmap='YlOrRd', vmax = int(tx90.max()))
    ax[i].set_title(months[i])
for i in [3,7,11]:
    fig.colorbar(cb, ax = ax[i], label = 'Tmax90')
plt.suptitle('Tmax Monthly 90th Percentile from 1983-2016', fontweight = 'bold')
plt.tight_layout()

In [ ]:
sellonlat(df_read(path,tmax,'Tmax')).groupby('time.month').mean('time').plot(label = 'Mean Tmax', alpha = 0.2)
sellonlat(tx90).plot(label = 'Tmax90', alpha = 0.7)
sellonlat(ctx90.groupby('time.month').mean('time')).plot(label='Average Peaks')
plt.legend()
plt.title('Monthly Averaged')

In [ ]:
selone(sellonlat(df_read(path,tmax,'Tmax'))).plot(label = 'Tmax', alpha = 0.2)
# sellonlat(tx90).plot(label = 'Tmax90', alpha = 0.7)
selone(sellonlat(ds)).plot(label='Average Peaks')
plt.legend()
plt.title('2010 - (monthly percentile)')

In [ ]:
# tx = CTX(ds_tx,ds_tx90).groupby('time.year').count('time')
# tx = tx.where(tx>0)


In [ ]:
# ctx90.groupby('time.year').count('time')

##### Analysis for Minimum Temperature

In [ ]:
######## Plotting Tmax 90 ########
fig, axes = plt.subplots(ncols = 4, nrows=3, figsize = (12,8))
ax=axes.flatten()
months = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG', 'SEP','OCT','NOV','DEC']
for i, month in enumerate(tn90['month']):
    cb = tn90.sel(month=month).plot(ax=ax[i], add_colorbar = False, cmap='YlOrRd', vmax = int(tn90.max()))
    ax[i].set_title(months[i])
for i in [3,7,11]:
    fig.colorbar(cb, ax = ax[i], label = 'Tmin90')
plt.suptitle('Tmin Monthly 90th Percentile from 1983-2016', fontweight = 'bold')
plt.tight_layout()

In [ ]:
sellonlat(df_read(path,tmin,'Tmin')).groupby('time.month').mean('time').plot(label = 'Mean Tmin', alpha = 0.2)
sellonlat(tn90).plot(label = 'Tmin90', alpha = 0.7)
sellonlat(ctn90.groupby('time.month').mean('time')).plot(label='Average Peaks')
plt.legend()
plt.title('Monthly Averaged')

#### Calculating for heatwave Number

In [ ]:
%%time

# Create subplots
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

# Plot data
vmax=8
cc='gist_heat_r'

hwdtx = hwn_tx.sum('time')/33
cb1 = hwdtx.where(hwdtx>0).plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)
ax[0].set_title('CTx90 1983-2016')

hwdtn = (hwn_tn.sum('time')/33)
hwdtn.where(hwdtn>0).plot(ax=ax[1], cmap=cc, vmax=vmax, add_colorbar=False)
ax[1].set_title('CTn90 1983-2016')

cm = 'RdBu_r'
vm = 0.5

txsl = hwn_tx.groupby('time.year').sum('time')
cb2 = sens_slopes(txsl.where(txsl>0).load()).plot(ax=ax[2], vmax=vm, vmin=-0.5, cmap=cm, add_colorbar=False)

tnsl = hwn_tn.groupby('time.year').sum('time')
sens_slopes(tnsl.where(tnsl>0).load()).plot(ax=ax[3], vmax=vm, vmin=-0.5, cmap=cm, add_colorbar=False)

label = ['HWN','Trends']

cb = [cb1, cb2]
# Add colorbars
for i, j in enumerate([0.62, 0.13]):
    cax = fig.add_axes([0.91, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label = label[i])

fig.tight_layout()
plt.savefig(path+'/figures/Climatologies of Yearly Heatwaves-using monthly percentiles.jpeg', bbox_inches='tight')

In [ ]:
hwdtx = hwn_tx.sel(time=slice(t1[i],t2[i])).sum('time')/10
hwdtx.where(hwdtx>0).plot()
# ax[0].set_title(f'CTx90 {t1[i]}-{t2[i]}')

In [ ]:
%%time
t1 = ['1984', '1994', '2004']
t2 = ['1993', '2003', '2013']
for i, j in enumerate(t1): 
    # Create subplots
    fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
    ax = ax.flatten()
    set_fig_params(ax)

    # Plot data
    vmax=12
    cc='coolwarm'

    hwdtx = hwn_tx.sel(time=slice(t1[i],t2[i])).sum('time')/10
    cb1 = hwdtx.where(hwdtx>0).plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)
    ax[0].set_title(f'CTx90 {t1[i]}-{t2[i]}')
    # break

    hwdtn = hwn_tn.sel(time=slice(t1[i],t2[i])).sum('time')/10
    hwdtn.where(hwdtn>0).plot(ax=ax[1], cmap=cc, vmax=vmax, add_colorbar=False)
    ax[1].set_title(f'CTn90 {t1[i]}-{t2[i]}')

    ds_gtehf.where(ds_gtehf>0).sel(year=slice(t1[i],t2[i])).mean('year').plot(ax=ax[2], cmap=cc, vmax=vmax, add_colorbar=False)
    ax[2].set_title(f'EHF {t1[i]}-{t2[i]}')

    cm = 'RdBu_r'
    vm = 1

    txsl = hwn_tx.groupby('time.year').sum('time').sel(year=slice(t1[i],t2[i]))
    # txsl = txsl.where(txsl>0).load()
    cb2 = sens_slopes(txsl.where(txsl>0).load()).plot(ax=ax[3], vmax=vm, cmap=cm, add_colorbar=False)
    ax[3].set_title(f'CTx90 {t1[i]}-{t2[i]}')


    tnsl = hwn_tn.groupby('time.year').sum('time').sel(year=slice(t1[i],t2[i]))
    sens_slopes(tnsl.where(tnsl>0).load()).plot(ax=ax[4], vmax=vm, cmap=cm, add_colorbar=False)
    ax[4].set_title(f'CTn90 {t1[i]}-{t2[i]}')

    sens_slopes(ds_gtehf.where(ds_gtehf>0).sel(year=slice(t1[i],t2[i])).load()).plot(ax=ax[5], cmap=cm, vmax=vm, add_colorbar=False)
    ax[5].set_title(f'EHF {t1[i]}-{t2[i]}')

    # # cb = [cb1, cb2]
    label = ['HWN','Trends']

    # cax2 = fig.add_axes([1,0.62,0.02,0.3])
    # fig.colorbar(cb3,cax = cax2, orientation='vertical', extend='both',label='ehf')
    cb = [cb1, cb2]
    # Add colorbars
    for i, j in enumerate([0.62, 0.13]):
        cax = fig.add_axes([1, j, 0.02, 0.3])
        fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label = label[i])

    fig.tight_layout()
    plt.savefig(f'{path}/figures/Climatologies of Yearly Heatwaves {t1[i]}-{t2[i]}_method_2.jpeg', bbox_inches='tight')

#### Lenght of the longest heatwaves (hwd)

In [ ]:
sens_slopes(hwn_tx.groupby('time.year').sum('time').load()).plot()

In [ ]:
%%time

# Create subplots
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

vmax = 20
vmax1 = 0.3
vmin1 = -0.3
# windows = candidates.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
# heatwave_starts = rising_filter_dask(windows, dim='rolling_dim')
cm = 'YlOrRd'
cc = 'rainbow'

hdtx = hwn_tx.groupby('time.year').sum('time').max('year')
cb1 = hdtx.where(hdtx>0).plot(ax=ax[0], cmap=cm, add_colorbar=False, vmax=vmax)
ax[0].set_title('CTx90 \n 1983-2016')

hdtn = hwn_tn.groupby('time.year').sum('time').max('year')
hdtn.where(hdtn>0).plot(ax=ax[1], cmap=cm, add_colorbar=False, vmax=vmax)
ax[1].set_title('CTn90 \n 1983-2016')


cb2 = sens_slopes((HWD(ctx90.load())).groupby('time.year').sum('time').load()).plot(ax=ax[2], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
sens_slopes((HWD(ctn90.load())).groupby('time.year').sum('time').load()).plot(ax=ax[3], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)

ax[2].set_title('CTx90pctl \n 1983-2016')
ax[3].set_title('CTc90pctl \n 1983-2016')

# Add colorbar
cb = [cb1, cb2]
labels = ['HWD','Trend']

for i, j in enumerate([0.585, 0.13]):
    cax = fig.add_axes([0.915, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label=labels[i])

fig.tight_layout()
plt.savefig(path+'/figures/lenght of the longest heatwave_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
ehf = df_read(path,tmean_ehf,'Tmax')

In [ ]:
ehf = df_read(path,tmean_ehf,'Tmax')
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(7, 7), subplot_kw={'projection': crs.PlateCarree()})
ehf.groupby('time.year').sum('time').idxmax('year').plot(ax=ax[0], cmap='YlOrRd', add_colorbar=True)
# cb = dx.plot(ax=ax, cmap='Set2', add_colorbar=False)
ehf.groupby('time.year').sum('time').argmax('year').plot(ax=ax[1], cmap='YlOrRd', add_colorbar=True)

for i in range(0,2):
    ax[i].set_extent([-20,15,4,24])
    ax[i].add_feature(feature.COASTLINE)
    ax[i].add_feature(feature.BORDERS)
    ax[i].add_feature(feature.STATES, linewidth = 0.2)
    ax[i].set_xticks([-20,-10,0,10], crs=crs.PlateCarree())
    ax[i].set_yticks([5,10,15,20], crs=crs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax[i].xaxis.set_major_formatter(lon_formatter)
    ax[i].yaxis.set_major_formatter(lat_formatter)
# cbar = fig.colorbar(cb , orientation ='horizontal')
# cbar.ax[i].tick_params(rotation=45)

In [ ]:
ehf.where(ehf>0).groupby('time.year').sum('time').argmax('year').plot(cmap='YlOrRd', add_colorbar=True)

In [ ]:
tm = df_read(path, tmean, 'Tmax')

In [ ]:
# tm.sel(latitude=6.6, longitude=-1.6, method='nearest').plot(alpha=0.3)
region = ehf.sel(latitude=6.6, longitude=-1.6, method='nearest')#,'1999'))
region.where(
    region.rolling(time=10).count()>=2
).plot(marker='*', alpha=0.3)
# region.where(
#     region.rolling(time=23).count()>=23
# ).plot(marker='.')
# plt.title('lenght of the longest heatwave at Ghana')
plt.grid()
# plt.savefig(path+'/figures/lenght_hwd_tn_gh_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
ehf.sel(longitude=-1.5, latitude=6.6, method= 'nearest').sel(time=slice('1998-01','1998-04')).plot()
tm.sel(longitude=-1.5, latitude=6.6, method= 'nearest').sel(time=slice('1998-01','1998-04')).plot()

In [ ]:
ehf

In [ ]:
tx = df_read(path, tmax, 'Tmax')

In [ ]:
dx = hwn_tx.groupby('time.year').sum('time').idxmax('year').load().astype(int)#.plot(ax = ax,cmap='Set2')

In [ ]:
dx.where((dx>=0)).plot()

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(7, 7), subplot_kw={'projection': crs.PlateCarree()})
# cb = ax.contourf(ds,levels= np.linspace(1980,2016,1), cmap='rainbow')
cb = dx.plot(ax=ax, cmap='Set2', add_colorbar=False)
ax.set_extent([-17.5,15,4,24])
ax.add_feature(feature.COASTLINE)
ax.add_feature(feature.BORDERS)
ax.add_feature(feature.STATES, linewidth = 0.2)
ax.set_xticks([-17,-10,0,10], crs=crs.PlateCarree())
ax.set_yticks([5,10,15,20], crs=crs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
cbar = fig.colorbar(cb , orientation ='horizontal')
cbar.ax.tick_params(rotation=45)
plt.savefig(path+'/figures/years_of_longest heatwave_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
f = ctx90.chunk({'time':1000}).rolling(time=16).count()

In [ ]:
d = f.compute()

In [ ]:
ds = xr.where(f>=16,1,0).compute()

In [ ]:
ds.max('time').plot()

In [ ]:
tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2013-05-15', '2013-07-01')).plot(alpha=0.3)
region = ctx90.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2013-05-15', '2013-07-01'))#,'1999'))
region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.3)
region.where(
    region.rolling(time=17).count()>=17
).plot(marker='.')
plt.title('lenght of the longest heatwave')
plt.grid()
plt.savefig(path+'/figures/lenght_hwd_tx_egypt_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-10')).plot(alpha=0.3)
region = ctx90.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-10'))#,'1999'))
region.where(
    region.rolling(time=3).count()>=3
).plot(marker='*', alpha=0.3)
region.where(
    region.rolling(time=18).count()>=18
).plot(marker='.')
plt.title('lenght of the longest heatwave')
# plt.title('lenght of the longest heatwave')
plt.grid()
plt.savefig(path+'/figures/lenght_hwd_tx_gh_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
#### Find the specific year the maximum hwd occurres
ds = hwn_tn.groupby('time.year').sum('time').idxmax('year').load().astype(int)#.plot(ax = ax,cmap='Set2')

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(7, 7), subplot_kw={'projection': crs.PlateCarree()})
# cb = ax.contourf(ds,levels= np.linspace(1980,2016,1), cmap='rainbow')
cb = ds.where((ds>0)).plot(ax=ax, cmap='Set2', add_colorbar=False)
ax.set_extent([-17.5,15,4,24])
ax.add_feature(feature.COASTLINE)
ax.add_feature(feature.BORDERS)
ax.add_feature(feature.STATES, linewidth = 0.2)
ax.set_xticks([-17,-10,0,10], crs=crs.PlateCarree())
ax.set_yticks([5,10,15,20], crs=crs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
cbar = fig.colorbar(cb , orientation ='horizontal')
cbar.ax.tick_params(rotation=45)
plt.savefig(path+'/figures/years_of_longest heatwave_tn_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
tn = df_read(path, tmin, 'Tmin')

In [ ]:
c = hwn_tn.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1996','1996-03'))
cs = tn.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1996-01','1996-03'))

In [ ]:
fs = ctn90.chunk({'time':1000}).rolling(time=16).count()

In [ ]:
dsx = xr.where(fs>=16,1,0).compute()

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(7, 7), subplot_kw={'projection': crs.PlateCarree()})
dsx.max('time').plot(ax=ax, add_colorbar=False)
ax.set_extent([-18,20,4,25])
ax.add_feature(feature.BORDERS)
ax.add_feature(feature.STATES)
ax.add_feature(feature.COASTLINE)
ax.set_xticks([-17,-10,0,10], crs=crs.PlateCarree())
ax.set_yticks([5,10,15,20], crs=crs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)

In [ ]:
tn.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01')).plot(alpha=0.3)
region = ctn90.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01'))#,'1999'))
region.where(
    region.rolling(time=3).count()>=3
).plot(marker='*', alpha=0.3)
region.where(
    region.rolling(time=23).count()>=23
).plot(marker='.')
plt.title('lenght of the longest heatwave at Ghana')
plt.grid()
plt.savefig(path+'/figures/lenght_hwd_tn_gh_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
tn.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1999-07-09', '1999-08-01')).plot(alpha=0.3)
region = ctn90.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1999-07-09', '1999-08-01'))#,'1999'))
region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.3)
region.where(
    region.rolling(time=17).count()>=17
).plot(marker='.')
# plt.title('lenght of the longest heatwave at Ivory Coast')
# plt.grid()
# plt.twinx()
# data_1000_tn.r.sel(longitude=-5.7, latitude=7,method='nearest').sel(time=slice('1999-07-13T03','1999-07-29T03')).plot()
# plt.title(None)
plt.title('lenght of the longest heatwave at Ivory Coast')
plt.grid()
plt.savefig(path+'/figures/lenght_hwd_tn_iv_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
ds = xr.open_mfdataset('/media/kenz/1B8D1A637BBA134B/CHIRTS/adaptor.*')
d = ds.sel(level=950).drop(['r','sst','ssr','clwc'])

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(12, 5),sharex=True, subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)
speed = np.sqrt(d.u**2 + d.v**2) 
cmap='coolwarm'
let = 6
cb1 = tn.sel(time='1999-07-12').plot(ax=ax[0],cmap=cmap, add_colorbar=False)
# d.sel(time='1999-07-12T03').plot.streamplot(x="longitude", y="latitude", u="u", v="v", density=1.2, linewidth = 2, color='k',ax=ax[0])
QV1 = d.sel(time='1999-07-12T03').coarsen(latitude=let, longitude=let, 
                                    boundary='trim').mean().plot.quiver(x="longitude", 
                                                                        y="latitude", u="u", v="v", ax=ax[0])
cb2 = tn.sel(time='1999-07-17').plot(ax=ax[1],cmap=cmap, add_colorbar=False)
QV2 = d.sel(time='1999-07-17T03').coarsen(latitude=let, longitude=let, 
                                    boundary='trim').mean().plot.quiver(x="longitude", 
                                                                        y="latitude", u="u", v="v", ax=ax[1])
cb3=tn.sel(time='1999-07-22').plot(ax=ax[2],cmap=cmap, add_colorbar=False)
QV3 = d.sel(time='1999-07-22T03').coarsen(latitude=let, longitude=let, 
                                    boundary='trim').mean().plot.quiver(x="longitude", 
                                                                        y="latitude", u="u", v="v", ax=ax[2])
cb4 = tn.sel(time='1999-07-27').plot(ax=ax[3],cmap=cmap, add_colorbar=False)
QV4 = d.sel(time='1999-07-27T03').coarsen(latitude=let, longitude=let, 
                                    boundary='trim').mean().plot.quiver(x="longitude", 
                                                                        y="latitude", u="u", v="v", ax=ax[3])
cb = [cb1,cb2,cb3,cb4]
for i, j in enumerate([0.585, 0.13]):
    cax = fig.add_axes([0.915, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label='Temperature [$^0$C]')
# QV = [QV1,QV2,QV3,QV4]
titles = ['day1', 'day2', 'day3', 'day4']
for i, j in enumerate(titles):
    ax[i].set_title(titles[i])
for i, j in enumerate([0.585, 0.13]):
    cax = fig.add_axes([0.915, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label='Temperature [$^0$C]')

plt.quiverkey(QV1,  1.2, 0.525, 1, r'$1 \frac{m}{s}$', labelpos='W',
                   coordinates='data')
plt.tight_layout()
plt.suptitle('')
# plt.savefig(path+'/figures/temperature_&_wind_at heatwavedays_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
level = 1000
data_1000_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=level)
hdtn = hwn_tn.groupby('time.year').sum('time').max('year').sel(longitude=slice(-20,15), latitude=slice(0,18))
data_str_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-radiation.nc').str/3600*3
data_sst_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-radiation.nc').sst-273.15
tn = df_read(path, tmin, 'Tmin')

In [ ]:
data_str_tn

In [ ]:
# level =1000
def plt_maps(hd,data,temp,data_str,data_sst,dates,level,wind_thres,temp_thres, rad_thres, sst_thres,rh_thres):
    import matplotlib as mpl
    def show_axes():
        for i,j in enumerate(ax):
            ax[i].set_extent([-20,15,0,17])
            ax[i].add_feature(feature.COASTLINE)
            ax[i].add_feature(feature.BORDERS)
            ax[i].add_feature(feature.STATES, linewidth = 0.2)
            ax[i].set_xticks([-10,-5,0,5], crs=crs.PlateCarree())
            ax[i].set_yticks([5,10], crs=crs.PlateCarree())
            lon_formatter = LongitudeFormatter(zero_direction_label=True)
            lat_formatter = LatitudeFormatter()
            ax[i].xaxis.set_major_formatter(lon_formatter)
            ax[i].yaxis.set_major_formatter(lat_formatter)
    fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(15, 10),sharex=True, subplot_kw={'projection': crs.PlateCarree()})
    ax = ax.flatten()
    show_axes()
    cmap = mpl.cm.rainbow
    cm = 'pink_r'
    # norm = mpl.colors.Normalize(vmax=wind_thres[0], vmin=wind_thres[1])
    # norm = mpl.colors.Normalize(vmin=0, vmax=int(wind_speed.max()))
    norm = mpl.colors.Normalize(vmin=0, vmax=wind_thres)
    # dates = ['1999-07-12', '1999-07-17', '1999-07-22', '1999-07-27']
    days = ['Day 0', 'Day 5', 'Day 10', 'Day 15']

    for i in range(0,4):
        hwnp = hd.where(hd>0).plot(ax=ax[i],cbar_kwargs={'orientation': 'horizontal', 'label':'HWD'}, vmax = 18, cmap='bone_r')
        ax[i].set_title(days[i])

    for i, j in enumerate(dates):    
        rhp = data.r.sel(longitude=slice(-20,15), latitude=slice(17,0)).sortby('latitude').sel(time=f'{dates[i]}T03', method='nearest').plot.contour(
            ax=ax[i+4],add_colorbar=False, cmap= 'rainbow', vmin=rh_thres[1], vmax=rh_thres[0])
        temp = tn.sel(longitude=slice(-20,15), latitude=slice(0,17)).sel(time=f'{dates[i]}', method = 'nearest').plot(
            ax=ax[i+4],cmap = cm,cbar_kwargs={'orientation': 'horizontal','label':'Temperature [$^o$C]'},vmax=temp_thres[0], vmin=temp_thres[1])
        radp = data_str.sel(longitude=slice(-20,15), latitude=slice(17,0)).sortby('latitude').sel(
            time=f'{dates[i]}T03', method = 'nearest').plot(ax=ax[8+i],cbar_kwargs={
            'orientation': 'horizontal', 'label':'OLR [$Wm^2$C]'}, cmap= 'rainbow', vmax = rad_thres[0], vmin=rad_thres[1])
        sstp = data_sst.sel(longitude=slice(-20,15), latitude=slice(17,0)).sortby('latitude').sel(time=f'{dates[i]}T03', method = 'nearest').plot(
            ax=ax[8+i],add_colorbar=False, cmap = 'jet', vmax=sst_thres[0], vmin=sst_thres[1])
        # ds = data.sel(longitude=slice(-15,10), latitude=slice(15,0)).sortby('latitude').drop(
        #     ['z','r','t','w','vo']).sel(time=f'{dates[i]}T03', method='nearest')
        # strm = ax[i].streamplot(x=ds["longitude"],y=ds["latitude"], u=ds["u"], v=ds["v"], color=ds.u.values, linewidth=1, cmap='rainbow', density=0.6, norm=norm)
        ds = data.sel(longitude=slice(-20,15), latitude=slice(17,0)).sortby('latitude').drop(
            ['z','r','t','w','vo']).sel(time=f'{dates[i]}T03', method='nearest')
        wind_speed = np.sqrt(ds.u[::3, ::3] ** 2 + ds.v[::3, ::3] ** 2)
        # norm = mpl.colors.Normalize(vmin=0, vmax=int(wind_speed.max()))
        quiver = ax[i].quiver(ds.u[::3,::3].longitude, ds.u[::3,::3].latitude, 
                               ds.u[::3,::3], ds.v[::3,::3],wind_speed, scale=1200-int(level[:-3]),
                              cmap='rainbow',linewidth=1.50, norm=norm, headwidth=5, headlength=5)

    for i in range(0,12):
        ax[i].set_xlabel(None)
        ax[i].set_ylabel(None)

    cbmap = [rhp,sstp]#,strm]
    lb = ['Relative Humidity [%]','Sea Surface Temperature [$^o$C]']
    for i, j in enumerate([0.45, 0.13]):#, 0.18]):
        cax = fig.add_axes([1, j, 0.02, 0.15])
        fig.colorbar(cbmap[i], cax=cax, orientation='vertical', label = lb[i])

    cax = fig.add_axes([1, 0.77, 0.02, 0.15])
    # cbar = fig.colorbar(strm.lines, cax=cax, orientation='vertical', label = 'wind speed [ms$^-2$]')
    cbar = fig.colorbar(quiver, cax=cax, orientation='vertical', label='wind speed [ms$^-2$]')

    fig.suptitle(f'Pressure level = {level[:-3]} hpa', fontweight ='bold')
    plt.tight_layout()
    plt.savefig(path+f'/figures/temp_wind_radiation_rh_hwn_sst_monthly_pctl_{level}{dates[0]}_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
lon = [-20,20]
lat = [0,20]
level = '850 tn'
fig, ax = plt.subplots(ncols=4, nrows=1, figsize=(15, 10),sharex=True, subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
def show_axes():
    for i,j in enumerate(ax):
        ax[i].set_extent([-20,15,0,17])
        ax[i].add_feature(feature.COASTLINE)
        ax[i].add_feature(feature.BORDERS)
        ax[i].add_feature(feature.STATES, linewidth = 0.2)
        ax[i].set_xticks([-20,-10,0,10,20], crs=crs.PlateCarree())
        ax[i].set_yticks([10,20], crs=crs.PlateCarree())
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax[i].xaxis.set_major_formatter(lon_formatter)
        ax[i].yaxis.set_major_formatter(lat_formatter)
show_axes()
dates = ['1997-08-09', '1997-08-15', '1997-08-22', '1997-08-30']
data_1000_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=1000)

for i, j in enumerate(dates):
    # data_str_tn.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01'))
    # ds = data_1000_tn.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').drop(
            # ['z','r','t','w','vo']).sel(time=f'{dates[i]}T15', method='nearest')
    # wind_speed = np.sqrt(ds.u[::3, ::3] ** 2 + ds.v[::3, ::3] ** 2)
        # norm = mpl.colors.Normalize(vmin=0, vmax=int(wind_speed.max()))
    # norm = mpl.colors.Normalize(vmin=0, vmax=10)
    # quiver = ax[i].quiver(ds.u[::3,::3].longitude, ds.u[::3,::3].latitude, 
    #                       ds.u[::3,::3], ds.v[::3,::3],wind_speed, scale=1200-int(level[:-3]),
    # #                       cmap='rainbow',linewidth=1.50, headwidth=5, headlength=5, norm=norm)
    radp = rad_tn.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').sel(
            time=f'{dates[i]}T15', method = 'nearest').plot(ax=ax[i],cbar_kwargs={
            'orientation': 'horizontal', 'label':'OLR [$Wm^2$C]'}, cmap= 'rainbow')
        
    # temp = tn.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[0],lat[1])).sel(time=f'{dates[i]}', method = 'nearest').plot(
    #     ax=ax[i],cmap = 'pink_r',cbar_kwargs={'orientation': 'horizontal','label':'Temperature [$^o$C]'}, vmax=34, vmin=16)
    # rhp = data_850_tn.r.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').sel(time=f'{dates[i]}T15', method='nearest').plot.contour(
    #         ax=ax[i],add_colorbar=True, cmap= 'rainbow')
    # sstp = data_sst_tn.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').sel(time=f'{dates[i]}T03', method = 'nearest').plot(
    #         ax=ax[i],add_colorbar=True, cmap = 'jet')

# cax = fig.add_axes([1, 0.47, 0.02, 0.15])
# cbar = fig.colorbar(quiver, cax=cax, orientation='vertical', label='wind speed [ms$^-2$]')

In [ ]:
rad_tn

In [ ]:
data_1000_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=1000)
plt_maps(hdtn, data_1000_tn, tn, data_str_tn,data_sst_tn,dates = ['1997-08-09', '1997-08-15', '1997-08-22', '1997-08-30'],level = '1000 tn',
        wind_thres = 10,temp_thres = [30,18], rad_thres = [10,-180], sst_thres = [30,23], rh_thres=[120,0])

In [ ]:
data_950_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=950)
plt_maps(hdtn, data_950_tn, tn, data_str_tn,data_sst_tn,dates = ['1997-08-09', '1997-08-15', '1997-08-22', '1997-08-30'],level = '950 tn',
        wind_thres = 14,temp_thres = [30,18], rad_thres = [10,-180], sst_thres = [30,23], rh_thres=[120,0])

In [ ]:
data_850_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=850)
plt_maps(hdtn, data_850_tn, tn, data_str_tn,data_sst_tn,dates = ['1997-08-09', '1997-08-15', '1997-08-22', '1997-08-30'],level = '850 tn',
        wind_thres = 15,temp_thres = [30,18], rad_thres = [10,-180], sst_thres = [30,23], rh_thres=[120,0])

In [ ]:
data_500_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=500)
plt_maps(hdtn, data_500_tn, tn, data_str_tn,data_sst_tn,dates = ['1997-08-09', '1997-08-15', '1997-08-22', '1997-08-30'],level = '500 tn',
        wind_thres = 20,temp_thres = [30,18], rad_thres = [10,-180], sst_thres = [30,23], rh_thres=[120,0])

In [ ]:
data_200_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=200)
plt_maps(hdtn, data_200_tn, tn, data_str_tn,data_sst_tn,dates = ['1997-08-09', '1997-08-15', '1997-08-22', '1997-08-30'],level = '200 tn',
        wind_thres = 60,temp_thres = [30,18], rad_thres = [10,-180], sst_thres = [30,25], rh_thres=[120,0])

In [ ]:
data_1000_tn = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=1000)

In [ ]:
plt_maps(hdtn, data_1000_tn, tn, data_str_tn,data_sst_tn,dates = ['1999-07-10', '1999-07-16', '1999-07-21', '1999-07-26'],level = '1000 tn',
        wind_thres = 10,temp_thres = [29,20], rad_thres = [10,-180], sst_thres = [29,22], rh_thres=[104,56])

In [ ]:
data_950 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=950)

In [ ]:
plt_maps(hdtn, data_950, tn, data_str_tn,data_sst_tn,dates = ['1999-07-11', '1999-07-16', '1999-07-21', '1999-07-26'],level = '950 tn',
        wind_thres = 12,temp_thres = [29,20], rad_thres = [10,-180], sst_thres = [29,22], rh_thres=[104,56])

In [ ]:
data_850 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=850)
plt_maps(hdtn, data_850, tn, data_str_tn,data_sst_tn,dates = ['1999-07-10', '1999-07-17', '1999-07-22', '1999-07-27'],level = '850 tn',
        wind_thres = 11,temp_thres = [29,20], rad_thres = [10,-180], sst_thres = [29,22], rh_thres=[120,0])

In [ ]:
data_500 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=500)
plt_maps(hdtn, data_500, tn, data_str_tn,data_sst_tn,dates = ['1999-07-10', '1999-07-17', '1999-07-22', '1999-07-27'],level = '500 tn',
        wind_thres = 15,temp_thres = [29,20], rad_thres = [10,-180], sst_thres = [29,22], rh_thres=[120,0])

In [ ]:
data_200 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=200)
plt_maps(hdtn, data_200, tn, data_str_tn,data_sst_tn,dates = ['1999-07-10', '1999-07-17', '1999-07-22', '1999-07-27'],level = '200 tn',
        wind_thres = 35,temp_thres = [29,20], rad_thres = [10,-180], sst_thres = [29,22], rh_thres=[120,0])

In [ ]:
#### Line Plots

In [ ]:
data_1000_tn.r.sel(longitude=-5.7, latitude=7,method='nearest').sel(time=slice('1999-07-10T03','1999-07-28T03')).plot()

In [ ]:
#### Maximum Temperature

In [ ]:
# level = 1000
# data_1000_tx = xr.open_dataset('../Era_5/adaptor.mars.internal-1696568296.3407738-20356-11-6eb92cae-2150-478f-8f10-5ce4465adc91.nc').sel(level=level)
hdtx = hwn_tx.groupby('time.year').sum('time').max('year').sel(longitude=slice(-15,10), latitude=slice(0,15))
# data_str_x = xr.open_dataset('../Era_5/adaptor.mars.internal-radiation.nc').str/3600*3
# data_sst_tx = xr.open_dataset('../Era_5/adaptor.mars.internal-radiation.nc').sst-273.15
tx = df_read(path, tmax, 'Tmax')

In [ ]:
# level =1000
def plt_map(hd,data,temp,data_str,data_sst,dates,level,wind_thres,temp_thres, rad_thres, sst_thres,rh_thres):
    import matplotlib as mpl
    def show_axes():
        for i,j in enumerate(ax):
            ax[i].set_extent([-22,25,4,30])
            ax[i].add_feature(feature.COASTLINE)
            ax[i].add_feature(feature.BORDERS)
            ax[i].add_feature(feature.STATES, linewidth = 0.2)
            ax[i].set_xticks([-20,-10,0,10,20], crs=crs.PlateCarree())
            ax[i].set_yticks([10,20,27], crs=crs.PlateCarree())
            lon_formatter = LongitudeFormatter(zero_direction_label=True)
            lat_formatter = LatitudeFormatter()
            ax[i].xaxis.set_major_formatter(lon_formatter)
            ax[i].yaxis.set_major_formatter(lat_formatter)
    fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(15, 10),sharex=True, subplot_kw={'projection': crs.PlateCarree()})
    ax = ax.flatten()
    show_axes()
    cmap = mpl.cm.rainbow
    cm = 'pink_r'
    # norm = mpl.colors.Normalize(vmax=wind_thres[0], vmin=wind_thres[1])
    # norm = mpl.colors.Normalize(vmin=0, vmax=int(wind_speed.max()))
    norm = mpl.colors.Normalize(vmin=0, vmax=wind_thres)
    # dates = ['1999-07-12', '1999-07-17', '1999-07-22', '1999-07-27']
    days = ['Day 0', 'Day 5', 'Day 10', 'Day 15']

    for i in range(0,4):
        hwnp = hd.where(hd>0).plot(ax=ax[i],cbar_kwargs={'orientation': 'horizontal', 'label':'HWD'}, vmax = 18, cmap='bone_r')
        ax[i].set_title(days[i])
    lon = [-30,25]
    lat = [0,35]
    for i, j in enumerate(dates):    
        rhp = data.r.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').sel(time=f'{dates[i]}T15', method='nearest').plot.contour(
            ax=ax[i+4],add_colorbar=False, cmap= 'rainbow', vmin=rh_thres[1], vmax=rh_thres[0])
        temp = tx.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[0],lat[1])).sel(time=f'{dates[i]}', method = 'nearest').plot(
            ax=ax[i+4],cmap = cm,cbar_kwargs={'orientation': 'horizontal','label':'Temperature [$^o$C]'},vmax=temp_thres[0], vmin=temp_thres[1])
        radp = data_str.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').sel(
            time=f'{dates[i]}T15', method = 'nearest').plot(ax=ax[8+i],cbar_kwargs={
            'orientation': 'horizontal', 'label':'OLR [$Wm^2$C]'}, cmap= 'rainbow', vmax = rad_thres[0], vmin=rad_thres[1])
        sstp = data_sst.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').sel(time=f'{dates[i]}T15', method = 'nearest').plot(
            ax=ax[8+i],add_colorbar=False, cmap = 'jet', vmax=sst_thres[0], vmin=sst_thres[1])
        # ds = data.sel(longitude=slice(-15,10), latitude=slice(15,0)).sortby('latitude').drop(
        #     ['z','r','t','w','vo']).sel(time=f'{dates[i]}T15', method='nearest')
        # strm = ax[i].streamplot(x=ds["longitude"],y=ds["latitude"], u=ds["u"], v=ds["v"], color=ds.u.values, linewidth=1, cmap='rainbow', density=0.6, norm=norm)
        ds = data.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').drop(
            ['z','r','t','w','vo']).sel(time=f'{dates[i]}T15', method='nearest')
        wind_speed = np.sqrt(ds.u[::3, ::3] ** 2 + ds.v[::3, ::3] ** 2)
        # norm = mpl.colors.Normalize(vmin=0, vmax=int(wind_speed.max()))
        quiver = ax[i].quiver(ds.u[::3,::3].longitude, ds.u[::3,::3].latitude, 
                               ds.u[::3,::3], ds.v[::3,::3],wind_speed, scale=1200-int(level[:-3]),
                              cmap='rainbow',linewidth=1.50, norm=norm, headwidth=5, headlength=5)

    for i in range(0,12):
        ax[i].set_xlabel(None)
        ax[i].set_ylabel(None)

    cbmap = [rhp,sstp]#,strm]
    lb = ['Relative Humidity [%]','Sea Surface Temperature [$^o$C]']
    for i, j in enumerate([0.45, 0.13]):#, 0.18]):
        cax = fig.add_axes([1, j, 0.02, 0.15])
        fig.colorbar(cbmap[i], cax=cax, orientation='vertical', label = lb[i])

    cax = fig.add_axes([1, 0.77, 0.02, 0.15])
    # cbar = fig.colorbar(strm.lines, cax=cax, orientation='vertical', label = 'wind speed [ms$^-2$]')
    cbar = fig.colorbar(quiver, cax=cax, orientation='vertical', label='wind speed [ms$^-2$]')

    fig.suptitle(f'Pressure level = {level[:-3]} hpa', fontweight ='bold')
    plt.tight_layout()
    plt.savefig(path+f'/figures/temp_wind_radiation_rh_hwn_sst_monthly_pctl_{level}_{dates[0]}_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
hdtx = hwn_tx.groupby('time.year').sum('time').max('year')
data_str_tx = xr.open_dataset('../Era_5/adaptor.mars.internal.tx.nc').str/3600*3
data_sst_tx = xr.open_dataset('../Era_5/adaptor.mars.internal.tx.nc').sst-273.15
tx = df_read(path, tmax, 'Tmax')

In [ ]:
lon = [-30,25]
lat = [-20,35]
level = '1000 tx'
data_tx_1000 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=1000)


In [ ]:
data_tx_1000 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=1000)
plt_map(hdtx, data_tx_1000, tx, data_str_tx,data_sst_tx,dates = ['2013-06-26', '2013-07-01', '2013-07-06', '2013-08-11'],level = '1000 tx',
        wind_thres = 16,temp_thres = [59,20], rad_thres = [10,-850], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
data_tx_950 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=950)
plt_map(hdtx, data_tx_950, tx, data_str_tx,data_sst_tx,dates = ['2013-06-26', '2013-07-01', '2013-07-06', '2013-08-11'],level = '950 tx',
        wind_thres = 20,temp_thres = [59,20], rad_thres = [10,-850], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
data_tx_850 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=850)
plt_map(hdtx, data_tx_850, tx, data_str_tx,data_sst_tx,dates = ['2013-06-26', '2013-07-01', '2013-07-06', '2013-08-11'],level = '850 tx',
        wind_thres = 20,temp_thres = [59,20], rad_thres = [10,-850], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
data_tx_500 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=500)
plt_map(hdtx, data_tx_500, tx, data_str_tx,data_sst_tx,dates = ['2013-06-26', '2013-07-01', '2013-07-06', '2013-08-11'],level = '500 tx',
        wind_thres = 23,temp_thres = [59,20], rad_thres = [10,-850], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
data_tx_200 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=200)
plt_map(hdtx, data_tx_200, tx, data_str_tx,data_sst_tx,dates = ['2013-06-26', '2013-07-01', '2013-07-06', '2013-08-11'],level = '200 tx',
        wind_thres = 38,temp_thres = [59,20], rad_thres = [10,-850], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
data_tx_950 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=950)


In [ ]:
hdtx = hwn_tx.groupby('time.year').sum('time').max('year').sel(longitude=slice(-15,10), latitude=slice(0,15))
data_str_tx = xr.open_dataset('../Era_5/adaptor.mars.internal.tx.nc').str/3600*3
data_sst_tx = xr.open_dataset('../Era_5/adaptor.mars.internal.tx.nc').sst-273.15
tx = df_read(path, tmax, 'Tmax')

In [ ]:
data_tx_1000 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=1000)
plt_map(hdtx, data_tx_950, tx, data_str_tx,data_sst_tx,dates = ['2015-12-16', '2015-12-23', '2015-12-28', '2016-01-03'],level = '1000 tx',
        wind_thres = 18,temp_thres = [38,15], rad_thres = [-100,-650], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
data_tx_950 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=950)
plt_map(hdtx, data_tx_950, tx, data_str_tx,data_sst_tx,dates = ['2015-12-16', '2015-12-23', '2015-12-28', '2016-01-03'],level = '950 tx',
        wind_thres = 22,temp_thres = [38,15], rad_thres = [-100,-650], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
tx = df_read(path, tmax, 'Tmax')
data_tx_950 = xr.open_dataset('../Era_5/adaptor.mars.internal-1697563853.3145142-15820-15-f77cdf85-8321-421e-80ef-f67af09303db.nc').sel(level=950)

In [ ]:
rh_tn = xr.open_dataset('/media/kenz/1B8D1A637BBA134B/Era_5/adaptor.mars.internal-rh_tn.nc').r.sel(level=1000)
rad_tn = xr.open_dataset('/media/kenz/1B8D1A637BBA134B/Era_5/adaptor.mars.internal-radiation_new_tn.nc').str/3600*3

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(14,6))
ax = fig.add_subplot(2,2,1)
# fig.subplots_adjust(right=0.75)
ax1 = fig.add_subplot(2,2,2)

twin1 = ax.twinx()
twin2 = ax.twinx()

twin2.spines.right.set_position(("axes", 1.2))

p1, = rad_tn.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1999-07-09', '1999-08-01')).plot(ax=ax,label = 'rad', color='k')
p2, = data_1000_tn.r.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1999-07-09', '1999-08-01')).plot(ax=twin1,alpha=0.5, label = 'rh')
p3, = tn.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1999-07-09', '1999-08-01')).plot(alpha=0.5, ax=twin2, label='tn', color= 'r')
# p4, = data_sst_tn.sel(latitude=5.1, longitude=-5.2, method='nearest').sel(time=slice('1999-07-09', '1999-08-01')).plot(ax=twin2,alpha=0.5,marker= '.',label='sst', color = 'r')
region = ctn90.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1999-07-09', '1999-08-01'))
p5, = region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.5, label='ctn90', ax=twin2, color = 'r')
p6, = region.where(
    region.rolling(time=18).count()>=18
).plot(marker='.', color='g', label='end', ax=twin2)

plt.grid()


# region2 = ctn90.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1999-07-09', '1999-08-01'))

twinax1 = ax1.twinx()
twinax2 = ax1.twinx()
twinax2.spines.right.set_position(("axes", 1.2))

p1a = rad_tn.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1998-07-12', '1998-07-30')).plot(ax=ax1,label = 'rad', color='k')
p2a, = rh_tn.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1998-07-12', '1998-07-30')).plot(ax=twinax1,alpha=0.5, label = 'rh')
p3a, = tn.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1998-07-12', '1998-07-30')).plot(alpha=0.5, ax=twinax2, label='tx', color= 'r')
region = ctn90.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1998-07-12', '1998-07-30'))
p6a, = region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.5, label='ctn90', ax=twinax2, color = 'r')

# tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2012-05-01', '2012-06-30')).plot(alpha=0.3, label='tx')
# region = ctn90.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1998-07-12', '1998-07-30'))
# region.where(region.rolling(time=1).count()>=1).plot(marker='*', alpha=0.3, label='ctx90', ax=twinax2)


# # ax.legend(handles=[p1, p2, p3, p4, p5,p6], loc='best')
# # ax1.legend(handles=[p3a], loc='best')

ax3 = fig.add_subplot(2,2,3)
twin3a = ax3.twinx()
twin3b = ax3.twinx()

twin3b.spines.right.set_position(("axes", 1.2))

p1b, = rad_tn.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01')).plot(ax=ax3,label = 'rad', color='k')
p2b, = rh_tn.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01')).plot(ax=twin3a,alpha=0.5, label = 'rh')
p3b, = tn.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01')).plot(alpha=0.5, ax=twin3b, label='tx', color= 'r')

# # p2c, = data_1000_tn.r.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01')).plot(ax=twin3a,alpha=0.5, label = 'rh')
# # p4c, = data_sst_tx.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01')).plot(ax=twin3b,alpha=0.5,marker= '.',label='sst', color = 'r')
region = ctn90.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01'))
p5b, = region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.5, label='ctn90', ax=twin3b, color = 'r')
p6b, = region.where(
    region.rolling(time=23).count()>=23
).plot(marker='.', color='g', label='end', ax=twin3b)
plt.grid()

# twin3a.set_title(None)
# # twin3b.set_title(None)
# # ax3.set_title('Ghana 1997')

# # ax3.legend(handles=[p1c, p2c, p3c, p4c, p5c,p6c], loc='best')

ax4 = fig.add_subplot(2,2,4)
twin4a = ax4.twinx()
twin4b = ax4.twinx()
twin4b.spines.right.set_position(("axes", 1.2))

p1c, = rad_tn.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('1999-08-09', '1999-09-01')).plot(ax=ax4,label = 'olr', color='k')
p2c, = rh_tn.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('1999-08-09', '1999-09-01')).plot(ax=twin4a,alpha=0.5, label = 'rh')
p3c, = tn.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('1999-08-09', '1999-09-01')).plot(alpha=0.5, label='tn', ax=twin4b, color='r')
region = ctn90.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('1999-08-09', '1999-09-01'))
p5c, = region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.5, label='ctn90', ax=twin4b)

ax4.set_title('Ghana 1999')
ax3.set_title('Ghana 19997')
ax1.set_title('Ivory Cost 1998')
ax.set_title('Ivory Coast 1999')

twin1.set_title(None)
twin2.set_title(None)
twinax1.set_title(None)
twinax2.set_title(None)
twin3a.set_title(None)
twin3b.set_title(None)
twin4a.set_title(None)
twin4b.set_title(None)

ax1.set(xlabel="Time [ Years ]", ylabel="OLR [ Wm$^-$$^2$ ]", ylim=(-135,-18))
ax.set(xlabel="Time [ Years ]", ylabel="OLR [ Wm$^-$$^2$ ]", ylim=(-135,-18))
ax3.set(xlabel="Time [ Years ]", ylabel="OLR [ Wm$^-$$^2$ ]", ylim=(-135,-18))
ax4.set(xlabel="Time [ Years ]", ylabel="OLR [ Wm$^-$$^2$ ]", ylim=(-135,-18))

twin1.yaxis.label.set_color(p2.get_color())
twin1.tick_params(axis='y', colors=p2.get_color())
twin1.set(ylim=(78,100))

twinax1.tick_params(axis='y', colors=p2a.get_color())
twinax1.yaxis.label.set_color(p2a.get_color())
twinax1.set(ylim=(78,100))

twin3a.tick_params(axis='y', colors=p2b.get_color())
twin3a.yaxis.label.set_color(p2b.get_color())
twin3a.set(ylim=(78,100))

twin4a.tick_params(axis='y', colors=p2c.get_color())
twin4a.yaxis.label.set_color(p2c.get_color())
twin4a.set(ylim=(78,100))

twin2.yaxis.label.set_color(p3.get_color())
twin2.tick_params(axis='y', colors=p3.get_color())
twin2.set(ylabel="Temperature [ $^0$C ]", ylim= (22,25.5))

twin3b.tick_params(axis='y', colors=p3c.get_color())
twin3b.yaxis.label.set_color(p3c.get_color())
twin3b.set(ylabel="Temperature [ $^0$C ]", ylim= (22,25.5))

twinax2.tick_params(axis='y', colors=p3b.get_color())
twinax2.yaxis.label.set_color(p3b.get_color())
twinax2.set(ylabel="Temperature [ $^0$C ]", ylim= (22,25.5))

twin4b.yaxis.label.set_color(p3d.get_color())
twin4b.tick_params(axis='y', colors=p3d.get_color())
twin4b.set(ylabel="Temperature [ $^0$C ]", ylim= (22,25.5))

fig.legend(handles=[p1, p2, p3, p5,p6],bbox_to_anchor=(1.07, 1.07))
plt.tight_layout()
plt.savefig(path+f'/figures/all_var_tn_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
# data_tx_950#.sel(latitude=5, longitude=6.5, method='nearest').sortby('time').plot(alpha=0.3, label = 'rh')
data = xr.open_dataset('/media/kenz/1B8D1A637BBA134B/Era_5/adaptor.mars.internal-1697563853.3145142-15820-15-f77cdf85-8321-421e-80ef-f67af09303db.nc')

In [ ]:
data

In [ ]:
tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2012-05-01', '2012-08-07')).plot(alpha=0.3, label='tx')
region = ctx90.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2012-05-01', '2012-08-07'))#,'1999'))
region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.3, label='ctx90')
#     region.rolling(time=18).count()>=18
# ).plot(marker='.', color='g')
# data_str_tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2013-06-08', '2013-06-30')).plot(alpha=0.3, label='str')
# data_tx_950

In [ ]:
str_tx = (xr.open_dataset('/media/kenz/1B8D1A637BBA134B/Era_5/adaptor.mars.internal-radiation_new.nc').str)/3600*3
rh_tx = xr.open_dataset('/media/kenz/1B8D1A637BBA134B/Era_5/adaptor.mars.internal-rh_new.nc').r.sel(level=1000)
str_tx
# str_tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2013-05-15', '2013-06-15')).plot()

In [ ]:
str_tx.sel(latitude=6.8, longitude=-5.2, method='nearest')#.sel(time=slice('1998-07-12', '1998-07-30'))

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(14,6))
ax = fig.add_subplot(2,2,1)
# fig.subplots_adjust(right=0.75)
ax1 = fig.add_subplot(2,2,2)

twin1 = ax.twinx()
twin2 = ax.twinx()

twin2.spines.right.set_position(("axes", 1.2))

p1, = data_str_tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2013-05-18', '2013-06-15')).plot(ax=ax,label = 'olr', color='k')
p2, = rh_tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2013-05-18', '2013-06-15')).plot(ax=twin1,alpha=0.5, label = 'rh')
p3, = tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2013-05-18', '2013-06-15')).plot(alpha=0.5, ax=twin2, label='tn', color= 'r')
# p4, = data_sst_tn.sel(latitude=5.1, longitude=-5.2, method='nearest').sel(time=slice('1999-07-09', '1999-08-01')).plot(ax=twin2,alpha=0.5,marker= '.',label='sst', color = 'r')
region = ctx90.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2013-05-18', '2013-06-15'))
p5, = region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.5, label='ctx90', ax=twin2, color = 'r')
p6, = region.where(
    region.rolling(time=18).count()>=18
).plot(marker='.', color='g', label='end', ax=twin2)

plt.grid()

twinax1 = ax1.twinx()
twinax2 = ax1.twinx()
twinax2.spines.right.set_position(("axes", 1.2))

p1a = str_tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2012-05-01', '2012-05-24')).plot(ax=ax1,label = 'olr', color='k')
p2a, = rh_tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2012-05-01', '2012-05-24')).plot(ax=twinax1,alpha=0.5, label = 'rh')
p3a, = tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2012-05-01', '2012-05-24')).plot(alpha=0.5, ax=twinax2, label='tx', color= 'r')
region = ctx90.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2012-05-01', '2012-05-24'))
p6a, = region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.5, label='ctx90', ax=twinax2, color = 'r')

# # tx.sel(latitude=20, longitude=15, method='nearest').sel(time=slice('2012-05-01', '2012-06-30')).plot(alpha=0.3, label='tx')
# # region = ctn90.sel(latitude=6.8, longitude=-5.2, method='nearest').sel(time=slice('1998-07-12', '1998-07-30'))
# # region.where(region.rolling(time=1).count()>=1).plot(marker='*', alpha=0.3, label='ctx90', ax=twinax2)


# # # ax.legend(handles=[p1, p2, p3, p4, p5,p6], loc='best')
# # # ax1.legend(handles=[p3a], loc='best')

ax3 = fig.add_subplot(2,2,3)
twin3a = ax3.twinx()
twin3b = ax3.twinx()

twin3b.spines.right.set_position(("axes", 1.2))

p1b, = str_tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-10')).plot(ax=ax3,label = 'olr', color='k')
p2b, = rh_tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-10')).plot(ax=twin3a,alpha=0.5, label = 'rh')
p3b, = tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-10')).plot(alpha=0.5, ax=twin3b, label='tx', color= 'r')

# # # p2c, = data_1000_tn.r.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01')).plot(ax=twin3a,alpha=0.5, label = 'rh')
# # # p4c, = data_sst_tx.sel(latitude=6.6, longitude=-1.6, method='nearest').sel(time=slice('1997-08-01', '1997-09-01')).plot(ax=twin3b,alpha=0.5,marker= '.',label='sst', color = 'r')
region = ctx90.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-10'))
p5b, = region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.5, label='ctx90', ax=twin3b, color = 'r')
p6b, = region.where(
    region.rolling(time=23).count()>=23
).plot(marker='.', color='g', label='end', ax=twin3b)
plt.grid()

# # twin3a.set_title(None)
# # # twin3b.set_title(None)
# # # ax3.set_title('Ghana 1997')

# # # ax3.legend(handles=[p1c, p2c, p3c, p4c, p5c,p6c], loc='best')

ax4 = fig.add_subplot(2,2,4)
twin4a = ax4.twinx()
twin4b = ax4.twinx()
twin4b.spines.right.set_position(("axes", 1.2))

p1c, = str_tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2014-10-22', '2014-11-22')).plot(ax=ax4,label = 'olr', color='k')
p2c, = rh_tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2014-10-22', '2014-11-22')).plot(ax=twin4a,alpha=0.5, label = 'rh')
p3c, = tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2014-10-22', '2014-11-22')).plot(alpha=0.5, label='tn', ax=twin4b, color='r')
region = ctx90.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2014-10-22', '2014-11-22'))
p5c, = region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.5, label='ctn90', ax=twin4b)

ax4.set_title('Ghana 2015')
ax3.set_title('Ghana 2014')
ax1.set_title('North Africa 2012')
ax.set_title('North Africa 2013')

twin1.set_title(None)
twin2.set_title(None)
twinax1.set_title(None)
twinax2.set_title(None)
twin3a.set_title(None)
twin3b.set_title(None)
twin4a.set_title(None)
twin4b.set_title(None)

ax1.set(xlabel="Time [ Years ]", ylabel="OLR [ Wm$^-$$^2$ ]", ylim=(-800,-450))
ax.set(xlabel="Time [ Years ]", ylabel="OLR [ Wm$^-$$^2$ ]", ylim=(-800,-450))
ax3.set(xlabel="Time [ Years ]", ylabel="OLR [ Wm$^-$$^2$ ]", ylim=(-450,-40))
ax4.set(xlabel="Time [ Years ]", ylabel="OLR [ Wm$^-$$^2$ ]", ylim=(-450,-40))

twin1.yaxis.label.set_color(p2.get_color())
twin1.tick_params(axis='y', colors=p2.get_color())
twin1.set(ylim=(0,15))

twinax1.tick_params(axis='y', colors=p2a.get_color())
twinax1.yaxis.label.set_color(p2a.get_color())
twinax1.set(ylim=(0,15))

twin3a.tick_params(axis='y', colors=p2b.get_color())
twin3a.yaxis.label.set_color(p2b.get_color())
twin3a.set(ylim=(10,100))

twin4a.tick_params(axis='y', colors=p2c.get_color())
twin4a.yaxis.label.set_color(p2c.get_color())
twin4a.set(ylim=(10,100))

twin2.yaxis.label.set_color(p3.get_color())
twin2.tick_params(axis='y', colors=p3.get_color())
twin2.set(ylabel="Temperature [ $^0$C ]", ylim= (39,48))

twin3b.tick_params(axis='y', colors=p3c.get_color())
twin3b.yaxis.label.set_color(p3c.get_color())
twin3b.set(ylabel="Temperature [ $^0$C ]", ylim= (28,38))

twinax2.tick_params(axis='y', colors=p3b.get_color())
twinax2.yaxis.label.set_color(p3b.get_color())
twinax2.set(ylabel="Temperature [ $^0$C ]", ylim= (39,48))

twin4b.yaxis.label.set_color(p3d.get_color())
twin4b.tick_params(axis='y', colors=p3d.get_color())
twin4b.set(ylabel="Temperature [ $^0$C ]", ylim= (28,38))

fig.legend(handles=[p1, p2, p3, p5,p6],bbox_to_anchor=(1.07, 1.07))
plt.tight_layout()
plt.savefig(path+f'/figures/all_var_tx_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
str_tx = xr.open_dataset('/media/kenz/1B8D1A637BBA134B/Era_5/adaptor.mars.internal-radiation_new.nc').str/3600*3

In [ ]:
dates = ['2012-05-01', '2012-05-06', '2012-05-12', '2012-05-18']
import matplotlib as mpl
def show_axes():
    for i,j in enumerate(ax):
        ax[i].set_extent([-22,25,4,30])
        ax[i].add_feature(feature.COASTLINE)
        ax[i].add_feature(feature.BORDERS)
        ax[i].add_feature(feature.STATES, linewidth = 0.2)
        ax[i].set_xticks([-20,-10,0,10,20], crs=crs.PlateCarree())
        ax[i].set_yticks([10,20,27], crs=crs.PlateCarree())
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax[i].xaxis.set_major_formatter(lon_formatter)
        ax[i].yaxis.set_major_formatter(lat_formatter)
fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(15, 10),sharex=True, subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
show_axes()
cmap = mpl.cm.rainbow
cm = 'pink_r'
# norm = mpl.colors.Normalize(vmax=wind_thres[0], vmin=wind_thres[1])
# norm = mpl.colors.Normalize(vmin=0, vmax=int(wind_speed.max()))
norm = mpl.colors.Normalize(vmin=0, vmax=15)
# dates = ['1999-07-12', '1999-07-17', '1999-07-22', '1999-07-27']
days = ['Day 0', 'Day 5', 'Day 10', 'Day 15']

for i in range(0,4):
    hwnp = hdtx.where(hdtx>0).plot(ax=ax[i],cbar_kwargs={'orientation': 'horizontal', 'label':'HWD'}, vmax = 18, cmap='bone_r')
    ax[i].set_title(days[i])
lon = [-30,25]
lat = [0,35]
for i, j in enumerate(dates):    
    rhp = rh_tx.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').sel(time=f'{dates[i]}T15', method='nearest').plot.contour(
        ax=ax[i+4],add_colorbar=False, cmap= 'rainbow', vmin=0, vmax=120)
    temp = tx.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[0],lat[1])).sel(time=f'{dates[i]}', method = 'nearest').plot(
        ax=ax[i+4],cmap = cm,cbar_kwargs={'orientation': 'horizontal','label':'Temperature [$^o$C]'},vmax=60, vmin=20)
    radp = data_str_tx.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').sel(
        time=f'{dates[i]}T15', method = 'nearest').plot(ax=ax[8+i],cbar_kwargs={
        'orientation': 'horizontal', 'label':'OLR [$Wm^2$C]'}, cmap= 'rainbow', vmax = -800, vmin=0)
    # sstp = data_sst.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').sel(time=f'{dates[i]}T15', method = 'nearest').plot(
    #     ax=ax[8+i],add_colorbar=False, cmap = 'jet', vmax=sst_thres[0], vmin=sst_thres[1])
    # ds = data.sel(longitude=slice(-15,10), latitude=slice(15,0)).sortby('latitude').drop(
    #     ['z','r','t','w','vo']).sel(time=f'{dates[i]}T15', method='nearest')
    # strm = ax[i].streamplot(x=ds["longitude"],y=ds["latitude"], u=ds["u"], v=ds["v"], color=ds.u.values, linewidth=1, cmap='rainbow', density=0.6, norm=norm)
    # ds = data.sel(longitude=slice(lon[0],lon[1]), latitude=slice(lat[1],lat[0])).sortby('latitude').drop(
    #     ['z','r','t','w','vo']).sel(time=f'{dates[i]}T15', method='nearest')
    # wind_speed = np.sqrt(ds.u[::3, ::3] ** 2 + ds.v[::3, ::3] ** 2)
    # # norm = mpl.colors.Normalize(vmin=0, vmax=int(wind_speed.max()))
    # quiver = ax[i].quiver(ds.u[::3,::3].longitude, ds.u[::3,::3].latitude, 
    #                        ds.u[::3,::3], ds.v[::3,::3],wind_speed, scale=1200-int(level[:-3]),
    #                       cmap='rainbow',linewidth=1.50, norm=norm, headwidth=5, headlength=5)

for i in range(0,12):
    ax[i].set_xlabel(None)
    ax[i].set_ylabel(None)

# cbmap = [rhp,sstp]#,strm]
# lb = ['Relative Humidity [%]','Sea Surface Temperature [$^o$C]']
# for i, j in enumerate([0.45, 0.13]):#, 0.18]):
#     cax = fig.add_axes([1, j, 0.02, 0.15])
#     fig.colorbar(cbmap[i], cax=cax, orientation='vertical', label = lb[i])

# cax = fig.add_axes([1, 0.77, 0.02, 0.15])
# # cbar = fig.colorbar(strm.lines, cax=cax, orientation='vertical', label = 'wind speed [ms$^-2$]')
# cbar = fig.colorbar(quiver, cax=cax, orientation='vertical', label='wind speed [ms$^-2$]')

fig.suptitle(f'Pressure level = {level[:-3]} hpa', fontweight ='bold')
plt.tight_layout()
# plt.savefig(path+f'/figures/temp_wind_radiation_rh_hwn_sst_monthly_pctl_{level}_{dates[0]}_1983-2016.jpeg', bbox_inches='tight')

### import matplotlib.pyplot as plt

fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(1,2,1)
# fig.subplots_adjust(right=0.75)
ax1 = fig.add_subplot(1,2,2)

twin1 = ax.twinx()
twin2 = ax.twinx()

twin2.spines.right.set_position(("axes", 1.2))

p1, = data_str_tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-05')).plot(ax=ax,label = 'rad', color='k')
p2, = data_tx_950.r.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-05')).plot(ax=twin1,alpha=0.5, label = 'rh')
p3, = tx.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-05')).plot(alpha=0.5, ax=twin2, label='tx', color= 'r')
p4, = data_sst_tx.sel(latitude=4, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-05')).plot(ax=twin2,alpha=0.5,marker= '.',label='sst', color = 'r')
region = ctx90.sel(latitude=5, longitude=6.5, method='nearest').sel(time=slice('2015-12-15', '2016-01-05'))#,'1999'))
p5, = region.where(
    region.rolling(time=1).count()>=1
).plot(marker='*', alpha=0.5, label='ctx90', ax=twin2, color = 'r')
p6, = region.where(
    region.rolling(time=18).count()>=18
).plot(marker='.', color='g', label='end', ax=twin2)

plt.grid()
ax.set(xlabel="Time [ Years ]", ylabel="OLR [ Wm$^-$$^2$ ]")
twin2.yaxis.label.set_color(p3.get_color())

ax.tick_params(axis='y', colors=p1.get_color())
twin1.tick_params(axis='y', colors=p2.get_color())
twin2.tick_params(axis='y', colors=p3.get_color())
ax.legend(handles=[p1, p2, p3, p4, p5,p6], loc='center')

ax.set_title(None)
plt.title(None)

plt.tight_layout()
plt.savefig(path+f'/figures/all_var_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
da = data.r.sel(latitude=5, longitude=6.5,level=950, method='nearest')
# da.resample(time

# da.sel(time=slice('1999-07-01T15', '1999-08-08T15'))

In [ ]:
da.sel(time='1999-07-01T03').values

In [ ]:
data_tx_850 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=850)
plt_map(hdtx, data_tx_850, tx, data_str_tx,data_sst_tx,dates = ['2015-12-16', '2015-12-23', '2015-12-28', '2016-01-03'],level = '850 tx',
        wind_thres = 22,temp_thres = [38,15], rad_thres = [-100,-650], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
data_tx_500 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=500)
plt_map(hdtx, data_tx_500, tx, data_str_tx,data_sst_tx,dates = ['2015-12-16', '2015-12-23', '2015-12-28', '2016-01-03'],level = '500 tx',
        wind_thres = 30,temp_thres = [38,15], rad_thres = [-100,-650], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
data_tx_200 = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(level=200)
plt_map(hdtx, data_tx_200, tx, data_str_tx,data_sst_tx,dates = ['2015-12-16', '2015-12-23', '2015-12-28', '2016-01-03'],level = '200 tx',
        wind_thres = 60,temp_thres = [38,15], rad_thres = [-110,-650], sst_thres = [32,18], rh_thres=[120,0])

In [ ]:
### Analysis Altitudewise
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15, 10),sharex=True, subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
lon = np.linspace(-5.7,-5.7)
lat = np.linspace(0,8.5)
lon1 = np.linspace(-20,-4)
lat2 = np.linspace(7,7)
show_axes()
for i in range(0,2):
    ax[i].set_extent([-20,10,0,15])
    ax[i].add_feature(feature.COASTLINE)
    ax[i].add_feature(feature.BORDERS)
    hwnp = hdtn.where(hdtn>0).plot(ax=ax[i],cbar_kwargs={'orientation': 'horizontal', 'label':'HWD'}, vmax = 18, cmap='bone_r')
ax[0].plot(lon,lat,linewidth=10, alpha=0.2, color= 'red')
ax[1].plot(lon1,lat2,linewidth=10, alpha=0.2, color= 'red')
plt.savefig(path+f'/figures/column_tn_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
### Maximum Temperature  ###
data = xr.open_dataset('/media/kenz/1B8D1A637BBA134B/Era_5/adaptor.mars.internal-1697563853.3145142-15820-15-f77cdf85-8321-421e-80ef-f67af09303db.nc')

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=2, figsize=(15, 8))
ax = ax.flatten()
dates = ['1999-07-10', '1999-07-17', '1999-07-22', '1999-07-27']
days = ['Day 0', 'Day 5', 'Day 10', 'Day 15']
for i, j in enumerate(dates):    
        rhp = data.r.sel(longitude=slice(-15,10), latitude=slice(15,0)).sel(
            time=f'{dates[i]}T03', method='nearest').sel(longitude=5.7, method='nearest').T.plot(
            ax=ax[i],add_colorbar=False if i<=2 else True, cmap= 'rainbow', vmin=0, vmax=120)
        ax[i].set_title(days[i])
        ax[i].invert_xaxis()
        tmp = (data.t.sel(longitude=slice(-15,10), latitude=slice(15,0)).sel(
            time=f'{dates[i]}T03', method='nearest')-273.15).sel(longitude=5.7, method='nearest').T.plot(
            ax=ax[i+4],add_colorbar=False if i<=2 else True, cmap= 'seismic', vmax=40)
        ax[i+4].set_title('')
        ax[i+4].invert_xaxis()
plt.tight_layout()
plt.savefig(path+f'/figures/rh_column_tn_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=2, figsize=(15, 8))
ax = ax.flatten()
dates = ['2015-12-16', '2015-12-23', '2015-12-28', '2016-01-03']
days = ['Day 0', 'Day 5', 'Day 10', 'Day 15']
for i, j in enumerate(dates):    
        rhp = data.r.sel(longitude=slice(-15,10), latitude=slice(15,0)).sel(
            time=f'{dates[i]}T15', method='nearest').sel(longitude=5.7, method='nearest').T.plot(
            ax=ax[i],add_colorbar=False if i<=2 else True, cmap= 'rainbow', vmin=0, vmax=110)
        ax[i].set_title(days[i])
        ax[i].invert_xaxis()
        tmp = (data.t.sel(longitude=slice(-15,10), latitude=slice(15,0)).sel(
            time=f'{dates[i]}T15', method='nearest')-273.15).sel(longitude=5.7, method='nearest').T.plot(
            ax=ax[i+4],add_colorbar=False if i<=2 else True, cmap= 'seismic', vmax=40)
        ax[i+4].set_title('')
        ax[i+4].invert_xaxis()
plt.tight_layout()
plt.savefig(path+f'/figures/rh_column_tx_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
### Analysis Altitudewise
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15, 10),sharex=True, subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
lon = np.linspace(-20,-4)
lat = np.linspace(7,7)
for i in range(0,2):
    ax[i].set_extent([-20,10,0,20])
    ax[i].add_feature(feature.COASTLINE)
    ax[i].add_feature(feature.BORDERS)
    hwnp = hdtn.where(hdtn>0).plot(ax=ax[i],cbar_kwargs={'orientation': 'horizontal', 'label':'HWD'}, vmax = 18, cmap='bone_r')
ax[0].plot(lon,lat,linewidth=10, alpha=0.2, color= 'red')

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=2, figsize=(15, 8))
ax = ax.flatten()
dates = ['1999-07-10', '1999-07-17', '1999-07-22', '1999-07-27']
days = ['Day 0', 'Day 5', 'Day 10', 'Day 15']
for i, j in enumerate(dates):    
        rhp = data.r.sel(longitude=slice(-15,10), latitude=slice(15,0)).sel(
            time=f'{dates[i]}T03', method='nearest').sel(latitude=7, method='nearest').plot(
            ax=ax[i],add_colorbar=False if i<=2 else True, cmap= 'rainbow', vmin=0, vmax=120)
        ax[i].set_title(days[i])
        ax[i].invert_yaxis()
        tmp = (data.t.sel(longitude=slice(-15,10), latitude=slice(15,0)).sel(
            time=f'{dates[i]}T03', method='nearest')-273.15).sel(latitude=7, method='nearest').plot(
            ax=ax[i+4],add_colorbar=False if i<=2 else True, cmap= 'seismic', vmax=40)
        ax[i+4].set_title('')
        ax[i+4].invert_yaxis()
plt.tight_layout()
plt.savefig(path+f'/figures/rh_column_tn_2_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=2, figsize=(15, 8))
ax = ax.flatten()
dates = ['2015-12-16', '2015-12-23', '2015-12-28', '2016-01-03']
days = ['Day 0', 'Day 5', 'Day 10', 'Day 15']
for i, j in enumerate(dates):    
        rhp = data.r.sel(longitude=slice(-15,10), latitude=slice(15,0)).sel(
            time=f'{dates[i]}T15', method='nearest').sel(latitude=7, method='nearest').plot(
            ax=ax[i],add_colorbar=False if i<=2 else True, cmap= 'rainbow', vmin=0, vmax=110)
        ax[i].set_title(days[i])
        ax[i].invert_yaxis()
        tmp = (data.t.sel(longitude=slice(-15,10), latitude=slice(15,0)).sel(
            time=f'{dates[i]}T15', method='nearest')-273.15).sel(latitude=7, method='nearest').plot(
            ax=ax[i+4],add_colorbar=False if i<=2 else True, cmap= 'seismic', vmax=40)
        ax[i+4].set_title('')
        ax[i+4].invert_yaxis()
plt.tight_layout()
plt.savefig(path+f'/figures/rh_column_tx_2_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, add_timestamp
from metpy.units import units

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.interpolate import cross_section
# data_tx = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(
    # time='2015-12-28T15', method ='nearest')


In [ ]:
ds = xr.open_dataset('../Era_5/adaptor.mars.internal-1696586869.7078497-9814-6-e617163c-d080-4831-bbee-0e77f388871a.nc').sel(
    time='2015-12-28T15', method ='nearest')

In [ ]:
data =  xr.open_dataset('/media/kenz/1B8D1A637BBA134B/Era_5/adaptor.mars.internal-1697563853.3145142-15820-15-f77cdf85-8321-421e-80ef-f67af09303db.nc'
                       ).sel(time='1999-01-01T03')
data = data.metpy.parse_cf().squeeze().sortby('latitude')

In [ ]:
print(list(data.variables))

In [ ]:
data

In [ ]:
isentlevs = [296.] * units.kelvin

In [ ]:
start = (-20.0, -30.0)
end = (35.0, 25.0)

In [ ]:
cross = cross_section(data, start, end).set_coords(('latitude', 'longitude'))

In [ ]:
cross

In [ ]:
data

In [ ]:
# cross_section(data, start, end).set_coords(('latitude', 'longitude'))


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.interpolate import cross_section

data = data.metpy.parse_cf().squeeze().sortby('latitude')
# Define the start and end points of the cross-section
start = (-10.0, -30.0)
end = (30.0, 25.0)
isentlevs = [296.] * units.kelvin
cross = cross_section(data, start, end).set_coords(('latitude', 'longitude'))
# Extract cross-section data
# cross = cross_section(data, start, end).set_coords(('latitude', 'longitude'))

# Calculate potential temperature, relative humidity, and wind components
cross['Relative_humidity'] = cross['r']
cross['u_wind'] = cross['u'].metpy.convert_units('knots')
cross['v_wind'] = cross['v'].metpy.convert_units('knots')
cross['t_wind'], cross['n_wind'] = mpcalc.cross_section_components(
    cross['u_wind'],
    cross['v_wind']
)

# Define the figure object and primary axes
fig = plt.figure(figsize=(10., 7.))
ax = plt.axes()

# Plot relative humidity using contourf
rh_contour = ax.contourf(cross['longitude'], cross['level'], cross['r'],
                         levels=np.arange(0, 101, 5), cmap='YlGnBu')
rh_colorbar = fig.colorbar(rh_contour)

# Plot potential temperature using contour
theta_contour = ax.contour(cross['longitude'], cross['level'], cross['t'],
                           levels=np.arange(250, 500, 5), colors='k', linewidths=2)
theta_contour.clabel(theta_contour.levels[1::2], fontsize=8, colors='k', inline=1,
                     inline_spacing=8, fmt='%i', rightside_up=True, use_clabeltext=True)

# Plot winds using the axes interface directly
wind_slc_vert = list(range(0, 5, 1))
wind_slc_horz = slice(0, 221, 20)
ax.barbs(cross['longitude'][wind_slc_horz], cross['level'][wind_slc_vert],
         cross['t_wind'][wind_slc_vert, wind_slc_horz],
         cross['n_wind'][wind_slc_vert, wind_slc_horz], color='k')

# Adjust the y-axis to be logarithmic
ax.set_yscale('symlog')
ax.set_yticklabels(np.arange(1000, 0, -100))
ax.set_ylim(cross['level'].max(), cross['level'].min())
ax.set_yticks(np.arange(1000, 200, -100))

# # Define the CRS and inset axes
# data_crs = ccrs.PlateCarree()
# ax_inset = fig.add_axes([0.125, 0.665, 0.25, 0.22], projection=data_crs)

# # # Plot geopotential height at 500 hPa using xarray's contour wrapper
# # ax_inset.contour(data['longitude'], data['latitude'], data['z'].sel(level=500),
# #                  levels=np.arange(5100, 6000, 60), cmap='inferno')

# Plot the path of the cross-section
# endpoints = data_crs.transform_points(ccrs.Geodetic(),
#                                       *np.vstack([start, end]).transpose()[::-1])
# ax_inset.scatter(endpoints[:, 0], endpoints[:, 1], c='k', zorder=2)
# ax_inset.plot(cross['longitude'], cross['latitude'], c='k', zorder=2)

# Add geographic features
# ax_inset.coastlines()
# ax_inset.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='k', alpha=0.2, zorder=0)

# Set the titles and axes labels
# ax_inset.set_title('')
ax.set_title(f'Cross-Section from {start} to {end}')
ax.set_ylabel('Pressure (hPa)')
ax.set_xlabel('Longitude (degrees east)')
rh_colorbar.set_label('Relative Humidity (%)')

plt.show()


In [ ]:
cross

In [ ]:
# Define the figure object and primary axes
fig = plt.figure(1, figsize=(16., 9.))
ax = plt.axes()

# Plot RH using contourf
rh_contour = ax.contourf(cross['longitude'], cross['level'], cross['Relative_humidity'],
                         levels=np.arange(0, 1.05, .05), cmap='YlGnBu')
rh_colorbar = fig.colorbar(rh_contour)

# # Plot potential temperature using contour, with some custom labeling
# theta_contour = ax.contour(cross['lon'], cross['isobaric'], cross['Potential_temperature'],
#                            levels=np.arange(250, 450, 5), colors='k', linewidths=2)
# theta_contour.clabel(theta_contour.levels[1::2], fontsize=8, colors='k', inline=1,
#                      inline_spacing=8, fmt='%i', rightside_up=True, use_clabeltext=True)

# # Plot winds using the axes interface directly, with some custom indexing to make the barbs
# # less crowded
# wind_slc_vert = list(range(0, 19, 2)) + list(range(19, 29))
# wind_slc_horz = slice(5, 100, 5)
# ax.barbs(cross['lon'][wind_slc_horz], cross['isobaric'][wind_slc_vert],
#          cross['t_wind'][wind_slc_vert, wind_slc_horz],
#          cross['n_wind'][wind_slc_vert, wind_slc_horz], color='k')

# # Adjust the y-axis to be logarithmic
# ax.set_yscale('symlog')
# ax.set_yticklabels(np.arange(1000, 50, -100))
# ax.set_ylim(cross['isobaric'].max(), cross['isobaric'].min())
# ax.set_yticks(np.arange(1000, 50, -100))

# # Define the CRS and inset axes
# data_crs = data['Geopotential_height'].metpy.cartopy_crs
# ax_inset = fig.add_axes([0.125, 0.665, 0.25, 0.25], projection=data_crs)

# # Plot geopotential height at 500 hPa using xarray's contour wrapper
# ax_inset.contour(data['x'], data['y'], data['Geopotential_height'].sel(isobaric=500.),
#                  levels=np.arange(5100, 6000, 60), cmap='inferno')

# # Plot the path of the cross section
# endpoints = data_crs.transform_points(ccrs.Geodetic(),
#                                       *np.vstack([start, end]).transpose()[::-1])
# ax_inset.scatter(endpoints[:, 0], endpoints[:, 1], c='k', zorder=2)
# ax_inset.plot(cross['x'], cross['y'], c='k', zorder=2)

# # Add geographic features
# ax_inset.coastlines()
# ax_inset.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='k', alpha=0.2, zorder=0)

# # Set the titles and axes labels
# ax_inset.set_title('')
# ax.set_title(f'NARR Cross-Section \u2013 {start} to {end} \u2013 '
#              f'Valid: {cross["time"].dt.strftime("%Y-%m-%d %H:%MZ").item()}\n'
#              'Potential Temperature (K), Tangential/Normal Winds (knots), Relative Humidity '
#              '(dimensionless)\nInset: Cross-Section Path and 500 hPa Geopotential Height')
# ax.set_ylabel('Pressure (hPa)')
# ax.set_xlabel('Longitude (degrees east)')
# rh_colorbar.set_label('Relative Humidity (dimensionless)')

# plt.show()

In [ ]:
isent_data['u'] = isent_data['u'].metpy.convert_units('kt')
isent_data['v'] = isent_data['v'].metpy.convert_units('kt')

In [ ]:
isent_data

In [ ]:
# Set up our projection and coordinates
crs = ccrs.PlateCarree()
lon = isent_data['pressure'].metpy.longitude
lat = isent_data['pressure'].metpy.latitude

# Coordinates to limit map area
bounds = [(-35., 30., -30., 40.)]
# Choose a level to plot, in this case 296 K (our sole level in this example)
level = 0

fig = plt.figure(figsize=(10., 8.))
# add_metpy_logo(fig, 120, 245, size='large')
ax = fig.add_subplot(1, 1, 1, projection=crs)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.75)
ax.add_feature(cfeature.STATES, linewidth=0.5)

# Plot the surface
clevisent = np.arange(0, 1000, 25)
cs = ax.contour(lon, lat, isent_data['pressure'].isel(isentropic_level=level),
                clevisent, colors='k', linewidths=1.0, linestyles='solid',
                transform=ccrs.PlateCarree())
cs.clabel(fontsize=10, inline=1, inline_spacing=7, fmt='%i', rightside_up=True,
          use_clabeltext=True)

# Plot RH
cf = ax.contourf(lon, lat, isent_data['r'].isel(isentropic_level=level),
                 range(10, 106, 5), cmap=plt.cm.gist_earth_r, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05,
                  extendrect='True')
cb.set_label('r', size='x-large')

# Plot wind barbs
ax.barbs(lon.values, lat.values, isent_data['u'].isel(isentropic_level=level).values,
         isent_data['v'].isel(isentropic_level=level).values, length=6,
         regrid_shape=20, transform=ccrs.PlateCarree())

# Make some titles
ax.set_title(f'{isentlevs[level]:~.0f} Isentropic Pressure (hPa), Wind (kt), '
             'Relative Humidity (percent)', loc='left')
add_timestamp(ax, isent_data['time'].values.astype('datetime64[ms]').astype('O'),
              y=0.02, high_contrast=True)
fig.tight_layout()

In [ ]:
isent_data.temperature[0].sortby('longitude').plot()

In [ ]:
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, add_timestamp
from metpy.units import units

In [ ]:
data = data.squeeze().set_coords(['longitude', 'latitude'])

In [ ]:
isent_data['u'] = isent_data['u'].metpy.convert_units('kt')
isent_data['v'] = isent_data['v'].metpy.convert_units('kt')

In [ ]:
temperature = tn.sel(time='1999-07-22')

In [ ]:
ds = ds.sel(time='1999-07-22')

In [ ]:
data = ds.squeeze().set_coords(['longitude', 'latitude'])

In [ ]:
d = d.sel(time='1999-07-22T03')

In [ ]:
d = d.load()

In [ ]:
float(wind_speed.max())

In [ ]:
import matplotlib.pyplot as plt

import metpy.calc as mpcalc
from metpy.cbook import example_data

# load example data
# ds = example_data()

# Calculate the total deformation of the flow
wind_speed = mpcalc.wind_speed(d.u, d.v)

# start figure and set axis
fig, ax = plt.subplots(figsize=(5, 5))

# plot wind speed
cf = ax.contourf(ds.longitude, ds.latitude, wind_speed, range(int(wind_speed.min()), int(wind_speed.max()), 1), cmap=plt.cm.BuPu)
plt.colorbar(cf, pad=0, aspect=50)
ax.barbs(d.longitude.values, d.latitude.values, d.u, d.v, color='black', length=0.01
         , alpha=0.5)
ax.set(xlim=(-27, 30), ylim=(-5, 30))
ax.set_title('Wind Speed Calculation')


In [ ]:
s

In [ ]:
c = s.sel(longitude=-6,latitude=6, method='nearest').to_dataframe()['Tmin']

In [ ]:
v = s.sel(longitude=-6,latitude=6, method='nearest')

In [ ]:
# Select rows which do not have NaN value in column 'Age'
# f
selected_rows = f[~f['Tmin'].isna()]

In [ ]:
selected_rows

In [ ]:
# tn.sel(longitude = -5.95, latitude = 6.15, method = 'nearest').sel(time = f'{year}').plot()

s.sel(longitude = -5.95, latitude = 6.15, method = 'nearest').sel(time = f'{year}').plot()
# ctn90.sel(longitude = -5.95, latitude = 6.15, method = 'nearest').sel(time = f'{year}').drop('month').plot()

In [ ]:
%%time

levels = range(int(tx.max())-50)
fig, axes = plt.subplots(ncols=1,figsize=[7,5], subplot_kw={'projection': crs.PlateCarree()})#ncols=1) #Creating the basis for the plot
axes.set_extent([-19,15,4,24])
axes.add_feature(feature.COASTLINE)
axes.add_feature(feature.BORDERS)
axes.add_feature(feature.STATES, linewidth = 0.2)
axes.set_xticks([-20,-10,0,10], crs=crs.PlateCarree())
axes.set_yticks([5,10,15,20], crs=crs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
axes.xaxis.set_major_formatter(lon_formatter)
axes.yaxis.set_major_formatter(lat_formatter)

def animate(time):
    tx.isel(year=time).plot(levels=levels, ax=axes, add_colorbar= False, cmap = 'plasma')

ani = animation.FuncAnimation(fig, animate, tx.year.size, interval=400, blit=False)

mld = tx.isel(year=0).plot.contourf(levels=levels, ax=axes, add_colorbar= False, cmap='plasma')
cbar= fig.colorbar(mld, shrink=0.6)
cbar.set_label('CTx90pctl')
fig.suptitle("Yearly Heatwave Days", fontsize= 18)

HTML(ani.to_jshtml())

In [ ]:
# tx_gt_tx90.resample(time='M').count('time').groupby('time.month').mean('time')[11].plot()
ani.save('figures/animated_avearged_yearly_heatwave_events.gif', writer='imagemagick', fps = 2) #Save animation as gif-file

In [ ]:
######## Plotting Tmax 90 ########
fig, axes = plt.subplots(ncols = 4, nrows=3, figsize = (12,8))
ax=axes.flatten()
set_fig_params(ax)
months = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG', 'SEP','OCT','NOV','DEC']
for i, month in enumerate(ds_tn90['month']):
    cb = ds_tn90.sel(month=month).plot(ax=ax[i], add_colorbar = False, cmap='YlOrRd', vmax = int(ds_tn90.max()))
    ax[i].set_title(months[i])
for i in [3,7,11]:
    fig.colorbar(cb, ax = ax[i], label = 'Tmin90')
plt.suptitle('Monthly 90th Percentile from 1983-2016', fontweight = 'bold')
plt.tight_layout()


In [ ]:
tn = CTX(ds_tn,ds_tn90).groupby('time.year').count('time')
tn = tn.where(tn>0)
# tn = tn.groupby('time.year').count('time')

In [ ]:
levels = range(int(tn.max())-50)
fig, axes = plt.subplots(figsize=[7,5])#ncols=1) #Creating the basis for the plot
def animate(time):
    tn.isel(year=time).plot(levels=levels, ax=axes, add_colorbar= False, cmap='magma')

ani = animation.FuncAnimation(fig, animate, tn.year.size, interval=400, blit=False)

mld= tn.isel(year=0).plot.contourf(levels=levels, ax=axes, add_colorbar= False,cmap='magma')
cbar= fig.colorbar(mld, extend = 'both')
cbar.set_label('CTx90pctl')
fig.suptitle("Yearly Heatwave Days", fontsize= 18)

HTML(ani.to_jshtml())

In [ ]:
ani.save('figures/Tmin_avearged_yearly_heatwave_events.gif', writer='imagemagick', fps = 2) #Save animation as gif-file

### Calculating EHI's

In [ ]:
 hwf_tx.where(hwf_tx>0).plot()

In [ ]:
 hwf_tn.where(hwf_tn>0).plot()

In [ ]:
%%time

fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

vmax = 35

cc = 'PuRd'

cb = hwf_tx.where(hwf_tx>0).plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)

hwf_tn.where(hwf_tn>0).plot(ax=ax[1], cmap=cc, vmax=vmax, add_colorbar=False)

ax[0].set_title('CTx90pct \n 1983-2016')
ax[1].set_title('CTn90pct \n 1983-2016')


cax = fig.add_axes([1,0.35,0.02,0.3])
fig.colorbar(cb, cax=cax, orientation='vertical', extend='both', label='HWF')

fig.tight_layout()
plt.savefig(path+'/figures/Yearly Number of Participating Heatwave Day-monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:

hwdtx = hwn_tx.sel(time=slice(t1,t2))#.sum('time')/33
# hwdtx.where(hwdtx>0).plot()

In [ ]:
hwdtn = hwn_tn.sel(time=slice(t1,t2)).sum('time')/33
hwdtn.where(hwdtn>0).plot()

In [ ]:
%%time
t1 = '1983'
t2 = '1992'
# Create subplots
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

# Plot data
vmax=2.5
cc='gist_heat_r'

hwdtx = hwn_tx.sel(time=slice(t1,t2)).sum('time')/33
cb1 = hwdtx.where(hwdtx>0).plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)
ax[0].set_title(f'CTx90 {t1}-{t2}')

hwdtn = (hwn_tn.sel(time=slice(t1,t2)).sum('time')/33)
hwdtn.where(hwdtn>0).plot(ax=ax[1], cmap=cc, vmax=vmax, add_colorbar=False)
ax[1].set_title(f'CTn90 {t1}-{t2}')

cm = 'RdBu_r'
vm = 1

txsl = hwn_tx.sel(time=slice(t1,t2)).groupby('time.year').sum('time')
cb2 = sens_slopes((txsl.where(txsl>0)).sel(year=slice(t1,t2)).load()).plot(ax=ax[2], vmax=vm, vmin=-1, cmap=cm, add_colorbar=False)

tnsl = hwn_tn.sel(time=slice(t1,t2)).groupby('time.year').sum('time')
sens_slopes((tnsl.where(tnsl>0)).sel(year=slice(t1,t2)).load()).plot(ax=ax[3], vmax=vm, vmin=-1, cmap=cm, add_colorbar=False)

label = ['HWN','Trends']

cb = [cb1, cb2]
# Add colorbars
for i, j in enumerate([0.61, 0.13]):
    cax = fig.add_axes([0.91, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label = label[i])

fig.tight_layout()
plt.savefig(f'{path}/figures/Climatologies of Yearly Heatwaves-using monthly percentiles {t1}-{t2}.jpeg', bbox_inches='tight')